<a href="https://colab.research.google.com/github/OxYz0n3/DBM_Week13_Activity/blob/master/DBM_Week13_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the required library
import sqlite3
import pandas as pd

# Connect to SQLite database (creates a new database if not exists)
conn = sqlite3.connect("company.db")

cursor = conn.cursor()

# Create "Departments" table
cursor.execute('''
  CREATE TABLE IF NOT EXISTS "Departments" (
    "DepartmentID" INTEGER NOT NULL,
    "DepartmentName" TEXT NOT NULL,
    PRIMARY KEY("DepartmentID")
  );
''')

# Create "Projects" table with foreign key constraint
cursor.execute('''
  CREATE TABLE IF NOT EXISTS "Projects" (
    "ProjectID" INTEGER NOT NULL,
    "ProjectName" TEXT NOT NULL,
    "DepartmentID" INTEGER NOT NULL,
    "HoursWorked" INTEGER NOT NULL DEFAULT 0,
    PRIMARY KEY("ProjectID"),
    CONSTRAINT "Departments_Projects" FOREIGN KEY ("DepartmentID") REFERENCES "Departments" ("DepartmentID")
  );
''')

# Create "Employees" table with foreign key constraint
cursor.execute('''
  CREATE TABLE IF NOT EXISTS "Employees" (
    "EmployeeID" INTEGER NOT NULL,
    "Name" TEXT NOT NULL,
    "Position" TEXT NOT NULL,
    "DepartmentID" INTEGER NOT NULL,
    "Salary" INTEGER NOT NULL,
    PRIMARY KEY("EmployeeID"),
    CONSTRAINT "Departments_Employees" FOREIGN KEY ("DepartmentID") REFERENCES "Departments" ("DepartmentID")
  );
''')

# Create "Assignments" table with foreign key constraints
cursor.execute('''
  CREATE TABLE IF NOT EXISTS "Assignments" (
    "EmployeeID" INTEGER NOT NULL,
    "ProjectID" INTEGER NOT NULL,
    "HoursWorked" INTEGER,
    CONSTRAINT "Employees_Assignments" FOREIGN KEY ("EmployeeID") REFERENCES "Employees" ("EmployeeID"),
    CONSTRAINT "Projects_Assignments" FOREIGN KEY ("ProjectID") REFERENCES "Projects" ("ProjectID")
  );
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
# 4.3 - Triggers
# Write an SQL trigger that automatically updates the total hours worked in a project whenever a new assignment is added.

# Connect to SQLite database (creates a new database if not exists)
conn = sqlite3.connect("company.db")

cursor = conn.cursor()

cursor.execute('''
  CREATE TRIGGER IF NOT EXISTS HoursWorkedTrigger
  AFTER INSERT ON Assignments
  BEGIN
    UPDATE Projects
    SET HoursWorked = (
        SELECT COALESCE(SUM(Assignments.HoursWorked), 0)
        FROM Assignments
        WHERE Assignments.ProjectID = NEW.ProjectID
    )
    WHERE ProjectID = NEW.ProjectID;
  END;
''')

conn.commit()
conn.close()

#Done by Cyprien SINGEZ


In [ ]:
# Connect to SQLite database (creates a new database if not exists)
conn = sqlite3.connect("company.db")

cursor = conn.cursor()

cursor.execute('''
    INSERT INTO "Departments" ("DepartmentID", "DepartmentName") VALUES
        (1, 'HR'),
        (2, 'Engineering'),
        (3, 'Marketing');
''')

# Insert data into "Projects" table
cursor.execute('''
    INSERT INTO "Projects" ("ProjectID", "ProjectName", "DepartmentID") VALUES
        (101, 'ProjectA', 2),
        (102, 'ProjectB', 2),
        (103, 'ProjectC', 3);
''')

# Insert data into "Employees" table
cursor.execute('''
    INSERT INTO "Employees" ("EmployeeID", "Name", "Position", "DepartmentID", "Salary") VALUES
        (1001, 'John Doe', 'Software Engineer', 2, 300),
        (1002, 'Jane Smith', 'HR Manager', 2, 400),
        (1003, 'Bob Johnson', 'Marketing Specialist', 3, 500),
        (1004, 'Alice Brown', 'HR Coordinator', 1, 350),
        (1005, 'Charlie Wilson', 'Software Developer', 2, 320),
        (1006, 'David Lee', 'Marketing Coordinator', 3, 450),
        (1007, 'Emma Davis', 'Recruitment Specialist', 1, 380),
        (1008, 'Frank White', 'System Architect', 2, 420),
        (1009, 'Grace Miller', 'Marketing Manager', 3, 550),
        (1010, 'Henry Johnson', 'Software Tester', 2, 340);
''')

# Insert data into "Assignments" table
cursor.execute('''
    INSERT INTO "Assignments" ("EmployeeID", "ProjectID", "HoursWorked") VALUES
        (1001, 101, 20),
        (1002, 103, 15),
        (1003, 102, 25),
        (1004, 101, 10),
        (1005, 102, 18),
        (1006, 103, 22),
        (1007, 101, 12),
        (1008, 102, 16),
        (1009, 103, 30),
        (1010, 101, 14);
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
con = sqlite3.connect("company.db")

cursor = con.cursor()

# Select the ProjectName with the Inner Join
running_total_employees = '''SELECT Name, ProjectName FROM Employees
   INNER JOIN
   Departments
   ON Employees.DepartmentID = Departments.DepartmentID
   INNER JOIN
   Projects
   ON Projects.DepartmentID = Departments.DepartmentID '''


running_total_employees = pd.read_sql_query(running_total_employees, con)
print(running_total_employees, end='\n\n')

# Select average salary, order it by maximum and join to have the ProjectName
running_avg = '''WITH total AS (
  SELECT
    AVG(Salary) AS AverSalary,
    DepartmentID
  FROM Employees
  GROUP BY DepartmentID
  )
  SELECT DepartmentName, AverSalary FROM total
  INNER JOIN Departments ON Departments.DepartmentID=total.DepartmentID
  ORDER BY AverSalary DESC LIMIT 1
  '''

# Commit the changes and close the connection

running_avg = pd.read_sql_query(running_avg, con)
print(running_avg)

con.commit()
con.close()

          Name ProjectName
0     John Doe    ProjectA
1     John Doe    ProjectB
2   Jane Smith    ProjectA
3   Jane Smith    ProjectB
4  Bob Johnson    ProjectC

  DepartmentName  AverSalary
0      Marketing       500.0


In [ ]:
# 4.2 Using a window function, rank employees in each department based on
# their salary.
# Done by Mahe Faure


# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('company.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

#Window Functions
#Objective: Rank the employees salary and group by department.

# Ranking
ranking_query = """
  SELECT
    Name,
    Salary,
    Departments.DepartmentName,
    RANK() OVER (PARTITION BY Departments.DepartmentName ORDER BY Salary DESC) AS
  RankEmployeesSalary
  FROM
    Employees
  JOIN
    Departments on Employees.DepartmentID = Departments.DepartmentID
"""

# Execute the query and load the result into a Pandas DataFrame
ranking_df = pd.read_sql_query(ranking_query, conn)
# Close the database connection
conn.close()
# Display the result
ranking_df

,Name,Salary,DepartmentName,RankEmployeesSalary
0,Frank White,420,Engineering,1
1,Jane Smith,400,Engineering,2
2,Henry Johnson,340,Engineering,3
3,Charlie Wilson,320,Engineering,4
4,John Doe,300,Engineering,5
5,Emma Davis,380,HR,1
6,Alice Brown,350,HR,2
7,Grace Miller,550,Marketing,1
8,Bob Johnson,500,Marketing,2
9,David Lee,450,Marketing,3


In [19]:
# Connect to SQLite database (creates a new database if not exists)
conn = sqlite3.connect("company.db")

query = "SELECT ProjectName, HoursWorked FROM Projects ORDER BY HoursWorked DESC"

df = pd.read_sql_query(query, conn)

conn.close()

df

#Done by Cyprien SINGEZ

,ProjectName,HoursWorked
0,ProjectA,30
1,ProjectB,25
2,ProjectC,15
